# Get IMDB, Amazon

In [ ]:
# from google.colab import drive
# drive.mount('/content/mydrive')

Drive already mounted at /content/mydrive; to attempt to forcibly remount, call drive.mount("/content/mydrive", force_remount=True).


In [ ]:
import pandas as pd
# ng = pd.read_csv('/content/mydrive/MyDrive/personal_data/Random_Data/name_gender.csv')
ng = pd.read_csv('name_gender.csv')

ng.columns = ['text','labels']
ng.reset_index(drop = True, inplace=True)
ng

,text,labels
0,Aaban&&,M
1,Aabha*,F
2,Aabid,M
3,Aabriella,F
4,Aada_,F
...,...,...
95020,Zyvion,M
95021,Zyvon,M
95022,Zyyanna,F
95023,Zyyon,M


# Concat IMDB, amzn, TSC

In [ ]:
def binarize(x):
    if x=='M':
        return 1
    else:
        return 0

In [ ]:
df_ = ng
# df_.reset_index(drop=True, inplace=True)
# df_['text_str']=df_['text']
df_['labels']=df_['labels'].apply(lambda x: binarize(x))
df_

,text,labels
0,Aaban&&,1
1,Aabha*,0
2,Aabid,1
3,Aabriella,0
4,Aada_,0
...,...,...
95020,Zyvion,1
95021,Zyvon,1
95022,Zyyanna,0
95023,Zyyon,1


# View DataSet we will be using as Test Set

In [ ]:
from collections import Counter
Counter(df_['labels'])

Counter({0: 60304, 1: 34721})

# innit Training Code

# 1. Activate GPU and Install Dependencies

In [ ]:
# Activate GPU for faster training by clicking on 'Runtime' > 'Change runtime type' and then selecting GPU as the Hardware accelerator
# Then check if GPU is available
import torch
torch.cuda.is_available()

True

In [ ]:
# Install required libraries
!pip install datasets transformers huggingface_hub
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


#2. Preprocess data

In [ ]:
# Set DistilBERT tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")#finiteautomata/beto-sentiment-analysis # "allenai/longformer-base-4096"

In [ ]:
df

,text,labels
0,Aaban&&,1
1,Aabha*,0
2,Aabid,1
3,Aabriella,0
4,Aada_,0
...,...,...
95020,Zyvion,1
95021,Zyvon,1
95022,Zyyanna,0
95023,Zyyon,1


In [ ]:
len_ = int(0.9*df.shape[0])
print("len_:",len_)

len_: 85522


In [ ]:

df = df_
df_null = df.isnull()
df_null
print(df_null[df_null['text']==True])
print(df_null[df_null['labels']==True])
len_ = int(0.9*df.shape[0])
print(len_)
df__train = df[:len]
df__test = df[len:]
df__test.reset_index(drop=True,inplace=True)

Empty DataFrame
Columns: [text, labels]
Index: []
Empty DataFrame
Columns: [text, labels]
Index: []
85522


In [ ]:
df__train

,text,labels
0,Aaban&&,1
1,Aabha*,0
2,Aabid,1
3,Aabriella,0
4,Aada_,0
...,...,...
85517,Tharan,1
85518,Tharel,1
85519,Tharen,1
85520,Tharin,1


In [ ]:
df__test

,text,labels
0,Tharron,1
1,Tharun,1
2,Thary,0
3,Tharyn,1
4,Thasha,0
...,...,...
9498,Zyvion,1
9499,Zyvon,1
9500,Zyyanna,0
9501,Zyyon,1


In [ ]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

# df = pd.DataFrame({'a': [0,1,2], 'b': [3,4,5]})
df_train = df__train
df_train
df_train.dropna(axis=0,inplace=True)
dataset = ds.dataset(pa.Table.from_pandas(df_train).to_batches())

### convert to Huggingface dataset
hg_dataset_train = Dataset(pa.Table.from_pandas(df_train))

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

# df = pd.DataFrame({'a': [0,1,2], 'b': [3,4,5]})
df_test = df__test
df_test
df_test.dropna(axis=0,inplace=True)
dataset = ds.dataset(pa.Table.from_pandas(df_test).to_batches())

### convert to Huggingface dataset
hg_dataset_test = Dataset(pa.Table.from_pandas(df_test))

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train_tsc = hg_dataset_train.map(preprocess_function, batched=True)
# tokenized_train_tsc.set_format(type='torch')# prevent ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length.

tokenized_test_tsc = hg_dataset_test.map(preprocess_function, batched=True)
# tokenized_test_tsc.set_format(type='torch')# prevent ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length.

  0%|          | 0/86 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [ ]:
# tokenized_train

In [ ]:
# Use data_collector to convert our samples to PyTorch tensors and concatenate them with the correct amount of padding
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 3. Training the model

In [ ]:
# Define DistilBERT as our base model:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifi

In [ ]:
# Define the evaluation metrics 
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [ ]:
# Log in to your Hugging Face account 
# Get your API token here https://huggingface.co/settings/token
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
# Define a new Trainer with all the objects we constructed so far
from transformers import TrainingArguments, Trainer

# repo_name = "TSC_finetuning-sentiment-movie-model-longformer"#Longformer
# repo_name = "TSC_SentimentA_IMDBAmznTSC" #Standard Bert
repo_name = "REA_GenderIdentification_v1" #Standard Bert

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch", 
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_tsc,
    eval_dataset=tokenized_test_tsc,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Cloning https://huggingface.co/malcolm/REA_GenderIdentification_v1 into local empty directory.


In [ ]:
# Train the model
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 85522
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 10692


Step,Training Loss
500,0.404800
1000,0.338100
1500,0.329200
2000,0.319900
2500,0.311600
3000,0.291200
3500,0.291100
4000,0.285200
4500,0.276000
5000,0.276300


Saving model checkpoint to REA_GenderIdentification_v1/checkpoint-5346
Configuration saved in REA_GenderIdentification_v1/checkpoint-5346/config.json
Model weights saved in REA_GenderIdentification_v1/checkpoint-5346/pytorch_model.bin
tokenizer config file saved in REA_GenderIdentification_v1/checkpoint-5346/tokenizer_config.json
Special tokens file saved in REA_GenderIdentification_v1/checkpoint-5346/special_tokens_map.json
tokenizer config file saved in REA_GenderIdentification_v1/tokenizer_config.json
Special tokens file saved in REA_GenderIdentification_v1/special_tokens_map.json
Saving model checkpoint to REA_GenderIdentification_v1/checkpoint-10692
Configuration saved in REA_GenderIdentification_v1/checkpoint-10692/config.json
Model weights saved in REA_GenderIdentification_v1/checkpoint-10692/pytorch_model.bin
tokenizer config file saved in REA_GenderIdentification_v1/checkpoint-10692/tokenizer_config.json
Special tokens file saved in REA_GenderIdentification_v1/checkpoint-10692

TrainOutput(global_step=10692, training_loss=0.262948856789032, metrics={'train_runtime': 554.2562, 'train_samples_per_second': 308.601, 'train_steps_per_second': 19.291, 'total_flos': 243797968013952.0, 'train_loss': 0.262948856789032, 'epoch': 2.0})

In [ ]:
import requests
def get_pass():
    
    pw = '1729173241:AAH-Xuwdpukn8bVu-nExWWUZsV5bw5cb360'
    chatid = '-516181344'
    
    dict_ = 'Code Passed'
    url1 = 'https://api.telegram.org/bot{}/sendMessage?chat_id={}&text={}&disable_web_page_preview=True&parse_mode=markdown'.format(
        pw, chatid, dict_)
    resp = requests.get(url1)


get_pass()

In [ ]:
# time.sleep(120)
# get_pass()
# time.sleep(120)
# get_pass()
# time.sleep(120)
# get_pass()

In [ ]:
# Compute the evaluation metrics
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9503
  Batch size = 16


{'epoch': 2.0,
 'eval_accuracy': 0.8798274229190782,
 'eval_f1': 0.852187419104323,
 'eval_loss': 0.33661723136901855,
 'eval_runtime': 7.3356,
 'eval_samples_per_second': 1295.464,
 'eval_steps_per_second': 80.975}

# 4. Analyzing new data with the model

In [ ]:
# # Upload the model to the Hub
trainer.push_to_hub()

Saving model checkpoint to REA_GenderIdentification_v1
Configuration saved in REA_GenderIdentification_v1/config.json
Model weights saved in REA_GenderIdentification_v1/pytorch_model.bin
tokenizer config file saved in REA_GenderIdentification_v1/tokenizer_config.json
Special tokens file saved in REA_GenderIdentification_v1/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/255M [00:00<?, ?B/s]

Upload file runs/Apr15_08-23-33_254a60576360/events.out.tfevents.1650011683.254a60576360.71.2: 100%|##########…

Upload file runs/Apr15_08-23-33_254a60576360/events.out.tfevents.1650011024.254a60576360.71.0:  50%|####9     …

To https://huggingface.co/malcolm/REA_GenderIdentification_v1
   58d0416..091ade5  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.8798274229190782}, {'name': 'F1', 'type': 'f1', 'value': 0.852187419104323}]}
To https://huggingface.co/malcolm/REA_GenderIdentification_v1
   091ade5..49ddc67  main -> main



'https://huggingface.co/malcolm/REA_GenderIdentification_v1/commit/091ade5729116e8a5b431701d0c4005979c66dea'

In [ ]:
# Run inferences with your new model using Pipeline
from transformers import pipeline

# sentiment_model = pipeline(model="federicopascual/finetuning-sentiment-model-3000-samples")
repo_name = "TSC_finetuning-sentiment-movie-model" #Standard Bert
repo_name = "TSC_SentimentA_IMDBAmznTSC" #Standard Bert
repo_name = "TSC_SentimentA_IMDBAmznTSC_2" #Standard Bert
repo_name = "REA_GenderIdentification_v1" #Standard Bert

sentiment_model = pipeline(model="malcolm/"+repo_name)

sentiment_model(["Alan",'Susan','Malcolm','Alex'])

Downloading:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

https://huggingface.co/malcolm/REA_GenderIdentification_v1/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpj_undbrf


Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

storing https://huggingface.co/malcolm/REA_GenderIdentification_v1/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/b057317853a053eb705e0e940834400923fdde8af75ee6287a37265893ffc27f.87afaa3ccede035a0be64e2a0b27db6d87dbb578dbcf4150274a7ad2e93d878a
creating metadata file for /root/.cache/huggingface/transformers/b057317853a053eb705e0e940834400923fdde8af75ee6287a37265893ffc27f.87afaa3ccede035a0be64e2a0b27db6d87dbb578dbcf4150274a7ad2e93d878a
loading configuration file https://huggingface.co/malcolm/REA_GenderIdentification_v1/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/b057317853a053eb705e0e940834400923fdde8af75ee6287a37265893ffc27f.87afaa3ccede035a0be64e2a0b27db6d87dbb578dbcf4150274a7ad2e93d878a
Model config DistilBertConfig {
  "_name_or_path": "malcolm/REA_GenderIdentification_v1",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/695k [00:00<?, ?B/s]

storing https://huggingface.co/malcolm/REA_GenderIdentification_v1/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/27899ac4e990834e596bfbc9e7eeb8fc54a1c94109c9b0a3986f1b58559cf96c.848c414913cfee271695b8761d3e947fb18a724fbad549de63228b20e5f2d615
creating metadata file for /root/.cache/huggingface/transformers/27899ac4e990834e596bfbc9e7eeb8fc54a1c94109c9b0a3986f1b58559cf96c.848c414913cfee271695b8761d3e947fb18a724fbad549de63228b20e5f2d615
https://huggingface.co/malcolm/REA_GenderIdentification_v1/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpzxber1bk


Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

storing https://huggingface.co/malcolm/REA_GenderIdentification_v1/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/9502c601aa8dae7b1432839fd1986213f4e5730f31d4cf60161e939a6452d3d4.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
creating metadata file for /root/.cache/huggingface/transformers/9502c601aa8dae7b1432839fd1986213f4e5730f31d4cf60161e939a6452d3d4.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
loading file https://huggingface.co/malcolm/REA_GenderIdentification_v1/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/eb35d65b8cda113dc07403bc84968a47b9ce43605ebed57b9440d2bc2f341897.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/malcolm/REA_GenderIdentification_v1/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/27899ac4e990834e596bfbc9e7eeb8fc54a1c94109c9b0a3986f1b58559cf96c.848c414913cfee271695b8761d3

[{'label': 'LABEL_1', 'score': 0.9925572872161865},
 {'label': 'LABEL_0', 'score': 0.9989209175109863},
 {'label': 'LABEL_1', 'score': 0.9970692992210388},
 {'label': 'LABEL_0', 'score': 0.9016308188438416}]

In [ ]:
sentiment_model(["Alan",'Susan','Malcolm','Bryan','Steward'])

[{'label': 'LABEL_1', 'score': 0.9925572872161865},
 {'label': 'LABEL_0', 'score': 0.9989209175109863},
 {'label': 'LABEL_1', 'score': 0.9970692992210388},
 {'label': 'LABEL_1', 'score': 0.9947145581245422},
 {'label': 'LABEL_1', 'score': 0.9873393774032593}]